# FIPI followup experiment with LLama3


FIPI text in prompt that we'll modify:

options for values are "strongly agree," "agree," "disagree," "strongly disagree," "neutral," and "unknown"

You agree that you are extraverted and enthusiastic. You agree that you are agreeable and kind. You agree that you are dependable and organized. You agree that you are emotionally stable and calm. You agree that you are open to experience and imaginative.

In [ ]:
import os
import replicate
os.environ["REPLICATE_API_TOKEN"] = ""

print(os.environ.get("REPLICATE_API_TOKEN"))


r8_baDLVQqQ4N8EbqCEVw3AXfCNnDYZ56e0DFvsw


In [19]:
import json
import re

# we're using the all_4 file and regular expressions to drop the necessary dimension.

dimensions = {
    "extraverted": "extraverted and enthusiastic",
    "agreeable": "agreeable and kind",
    "conscientiousness": "dependable and organized",
    "stable": "emotionally stable and calm",
    "open": "open to new experiences and imaginative"
}

fipiPrompts = {
    "open": "The degree to which an individual is creative and imaginative or conventional and grounded. Made up of imagination, artistic interests, emotionality, adventurousness, intellect, and liberalism facets.",
    "conscientiousness": "The measurement of an individual’s ability to control their impulses. Made up of self-efficacy, orderliness, dutifulness, achievement striving, self-discipline, and cautiousness facets.",
    "extraverted": "The extent to which someone is out-going and enjoys interacting with the external world. Made up of friendliness, gregariousness, assertiveness, activity level, excitement seeking, and cheerfulness facets.",
    "agreeable": "The measure of social harmony, non-confrontation & cooperation that an individual may pursue. Made up of trust, morality, altruism, cooperation, modesty, and sympathy facets.",
    "stable": "The extent to which someone reacts less emotionally and is less easily upset. They tend to be emotionally stable, calm, and do not constantly experience negative feelings. Made up of stress-resistant, positive attitude, and self-confident"
}

questionPrompt = "How much do you agree that the following characteristics describe you? To answer this question use only one number: write 1 if you disagree strongly, write 2 if you disagree a little, write 3 if you neither agree nor disagree, write 4 if you agree a little, write 5 if you strongly agree. Write only one number according to the instructions WITHOUT ANY ADDITIONAL TEXT. "

def load_jsonl(filepath):
    with open(filepath, 'r') as file:
        data = [json.loads(line) for line in file]
    return data

def update_message(messages, fipiDimension):
    pattern = f"You (agree|strongly agree|disagree|strongly disagree|unknown|neutral) that you are {dimensions[fipiDimension]}."
    message = messages[0]['content']

    newMessage = re.sub(
        pattern,
        "",
        message
    )
    messages[0]['content'] = newMessage
    messages.append({"role": "user", "content": f"{questionPrompt}: {fipiPrompts[fipiDimension]}"})

    return messages


# roles are system, user, and assistant
# first one is system
def generate_prompt_format(messages):
    prompt = "<|begin_of_text|>"

    for message in messages:
        role = message["role"]
        content = message["content"]
        prompt += f"<|start_header_id|>{role}<|end_header_id|>\n\n{content}<|eot_id|>"
    prompt += "<|start_header_id|>assistant<|end_header_id|>\n\n"
    return prompt


def call_llama_replicate(text):
    promptText = generate_prompt_format(text)
    result = replicate.run(
    "meta/meta-llama-3-70b-instruct",
    input={
        "top_p": 0.9,
        "prompt": promptText,
        "max_tokens":200,
        "min_tokens": 0,
        "temperature": 0.6,
        "prompt_template": "{prompt}",
        "presence_penalty": 1.15
    },
    )
    return("".join(result))

def perform_experiment(json_data):
    responses = []
    for idx in range(len(json_data)):
        system_messages = json_data[idx]

        messages = system_messages.copy()
        #print(f'this is the message for baseline ：{condition_file}')
        #print(messages)
        response = call_llama_replicate(messages)
        responses.append(response)

    return responses



In [13]:
df = load_jsonl('json_datasets/all_4.jsonl')


In [ ]:
# this will run the experiment for "extraverted"
# options for fipiDim variable are
#  
# extraverted
# agreeable
# conscientiousness
# stable
# open

fipiDim = "extraverted"

dfExtraverted = []
for row in df:
    dfExtraverted.append(update_message(row['messages'], fipiDim))

samples = dfExtraverted
results = perform_experiment(samples)


In [23]:
results

['4', '3', '3', '3', '3', '3', '3', '4', '3', '3']